# Exosystems

This program evaluates the deflection of light due to the mean exoplanet
where the mean values are taken from the catalogue. It also evalutes the light deflection for different combinations of masses and radius of the bodies.

## Libraries

In [20]:
import numpy as np
import pandas as pd
from astronavigation.planets import SolarSystem, Body
from astropy import constants
from astronavigation.deflection import *
from astronavigation.save_df import save_df

## Useful objects

In [2]:
# Define constants
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

# create solar system and jupiter
ss = SolarSystem()
jupiter = ss.getPlanet('jupiter')
sun = ss.getSun()

v_null = np.array([0, 0, 0])

## Catalogue

In [3]:
path = 'exo_archive.csv'
catalogue = pd.read_csv(path)
mean_val = catalogue.mean(numeric_only=True)
max_val = catalogue.max(numeric_only=True)
min_val = catalogue.min(numeric_only=True)
catalogue.head()

,pl_name,hostname,pl_orbsmax,pl_radj,pl_massj,st_rad,st_mass,ra,dec,sy_dist,sy_gaiamag
0,Proxima Cen b,Proxima Cen,0.0485,NaN,NaN,0.14,0.12,217.393466,-62.676182,1.30119,8.95361
1,eps Eri b,eps Eri,3.5000,NaN,NaN,0.76,0.81,53.228431,-9.458172,3.20260,3.47719
2,GJ 887 b,GJ 887,0.0680,NaN,NaN,0.47,0.49,346.466827,-35.853069,3.29000,NaN
3,GJ 887 c,GJ 887,0.1200,NaN,NaN,0.47,0.49,346.466827,-35.853069,3.29000,NaN
4,Ross 128 b,Ross 128,0.0496,NaN,NaN,0.20,0.17,176.937604,0.799290,3.37454,9.59067


In [4]:
mean_val

pl_orbsmax      9.777852
pl_radj         0.480629
pl_massj        2.158434
st_rad          1.485338
st_mass         0.956092
ra            240.580033
dec            21.631457
sy_dist       660.015026
sy_gaiamag     12.488446
dtype: float64

In [5]:
max_val

pl_orbsmax    7506.000000
pl_radj          6.900000
pl_massj        31.000000
st_rad          83.800000
st_mass         10.940000
ra             359.974984
dec             85.736533
sy_dist       8200.000000
sy_gaiamag      19.879000
dtype: float64

In [6]:
min_val

pl_orbsmax     0.004400
pl_radj        0.045000
pl_massj       0.000060
st_rad         0.110000
st_mass        0.000000
ra             0.185606
dec          -88.121111
sy_dist        1.301190
sy_gaiamag     2.926270
dtype: float64

## Mean body deflection

In [7]:
# now we estimate the deflection generated by the mean planet
# we center the coordinate system into the star of the exosystem
planet = Body(mass=mean_val['pl_massj']*jupiter.mass,
          pos=mean_val['pl_orbsmax']*AU*np.array([0, 1, 0]),
          radius=mean_val['pl_radj']*jupiter.radius)
# mean star
star = Body(mass=mean_val['st_mass']*sun.mass,
            pos=np.array([0, 0, 0]),
            radius=mean_val['st_rad']*sun.radius)

# observer
x_obs = AU*np.array([0, -1, 0])

# target
dist = mean_val['sy_dist'] * pc

for pl in [star, planet]:
    # impact angle
    chi = pl.radius / np.linalg.norm(x_obs - pl.pos)


    # direction
    l0 = -np.array([np.sin(chi), np.cos(chi), 0])
    x = -dist*l0 + x_obs

    # deflection
    dls = deflection(l0, x, pl.pos, x_obs, eps, v_null, pl.mass)
    dl1 = np.linalg.norm(dls)

    print('-------------------------------')
    print(f'deflection: {np.rad2deg(dl1)*3600*1e6} muas')
    print(f'dr: {dist*dl1/AU} AU\n')

-------------------------------
deflection: 1127538.6608991732 muas
dr: 744.1924582366564 AU

-------------------------------
deflection: 74721.74431447653 muas
dr: 49.31747399311012 AU



## Test different combinations

In [17]:
# evaluate deflection for different combinations of mass and radius
planet_masses = [0.05, 0.2, 0.5, 1, 2, 5, 10, 15, 20, 30]
planet_radii = [0.05, 0.2, 0.5, 1, 2, 3, 5, 7]

stellar_masses = [0.05, 0.2, 0.5, 1, 2, 5, 10]
stellar_radii = [0.1, 1, 5, 10, 20]

planets = np.zeros((len(planet_masses), len(planet_radii)))
stars = np.zeros((len(stellar_masses), len(stellar_radii)))

# planets
for i in range(len(planet_masses)):
    for j in range(len(planet_radii)):
        
        planet = Body(mass=planet_masses[i]*jupiter.mass,
                      pos=5.2*AU*np.array([0, 1, 0]),
                      radius=planet_radii[j]*jupiter.radius)
        # observer
        x_obs = AU*np.array([0, -1, 0])

        # target
        dist = mean_val['sy_dist'] * pc
        
        # impact angle
        chi = planet.radius / np.linalg.norm(x_obs - planet.pos)

        # direction
        l0 = -np.array([np.sin(chi), np.cos(chi), 0])
        x = -dist*l0 + x_obs

        # deflection
        dls = deflection(l0, x, planet.pos, x_obs, eps, planet.mass, v_null)
        dl1 = np.linalg.norm(dls)

        print(f'--- mass: {planet_masses[i]}, radius: {planet_radii[j]} ----------------------------')
        print(f'deflection: {np.rad2deg(dl1)*3600*1e6} muas')
        print(f'dr: {dist*dl1/AU} AU\n')
        planets[i, j] = np.round(np.rad2deg(dl1)*3600*1e6, 2)
        
# stars
for i in range(len(stellar_masses)):
    for j in range(len(stellar_radii)):
        
        planet = Body(mass=stellar_masses[i]*sun.mass,
                      pos=np.array([0, 0, 0]),
                      radius=stellar_radii[j]*sun.radius)
        # observer
        x_obs = AU*np.array([0, -1, 0])

        # target
        dist = mean_val['sy_dist'] * pc
        
        # impact angle
        chi = planet.radius / np.linalg.norm(x_obs - planet.pos)

        # direction
        l0 = -np.array([np.sin(chi), np.cos(chi), 0])
        x = -dist*l0 + x_obs

        # deflection
        dls = deflection(l0, x, planet.pos, x_obs, eps, planet.mass, v_null,)
        dl1 = np.linalg.norm(dls)

        print(f'---mass: {stellar_masses[i]}, radius: {stellar_radii[j]}----------------------------')
        print(f'deflection: {np.rad2deg(dl1)*3600*1e6} muas')
        print(f'dr: {dist*dl1/AU} AU\n')
        stars[i, j] = np.round(np.rad2deg(dl1)*3600*1e6, 2)

--- mass: 0.05, radius: 0.05 ----------------------------
deflection: 16638.668989213395 muas
dr: 10.981771540316162 AU

--- mass: 0.05, radius: 0.2 ----------------------------
deflection: 4159.667247257191 muas
dr: 2.745442885048576 AU

--- mass: 0.05, radius: 0.5 ----------------------------
deflection: 1663.8668987994831 muas
dr: 1.0981771539511893 AU

--- mass: 0.05, radius: 1 ----------------------------
deflection: 831.9334492151098 muas
dr: 0.549088576853735 AU

--- mass: 0.05, radius: 2 ----------------------------
deflection: 415.9667242382924 muas
dr: 0.27454428818314863 AU

--- mass: 0.05, radius: 3 ----------------------------
deflection: 277.31114908190295 muas
dr: 0.18302952518463353 AU

--- mass: 0.05, radius: 5 ----------------------------
deflection: 166.38668866138124 muas
dr: 0.10981771459084635 AU

--- mass: 0.05, radius: 7 ----------------------------
deflection: 118.84763391410041 muas
dr: 0.07844122415067555 AU

--- mass: 0.2, radius: 0.05 ----------------------

In [22]:
# save planets
rows = planet_masses
columns = planet_radii
data = planets.T
path = f'Data/exoplantes_latex'
save_df(data, columns, rows, path)

In [23]:
# save stars
rows = stellar_masses
columns = stellar_radii
data = stars.T
path = f'Data/exostars_latex'
save_df(data, columns, rows, path)